In [106]:
import pandas as pd
import numpy as np
import re
data = pd.read_csv('techscene.csv')

# Andmete puhastamine

In [107]:
for column in data.columns:#teeme kõik väiketähtedeks
  data[column]=data[column].str.lower()
#job_name
#eemaldame (m/f/d),(m/f/x),| SEB, Tallinn,
data['job_name']=data['job_name'].str.replace('(m/f/d)', '').replace('(m/f/x)','').replace('| SEB, Tallinn','').replace('\"','')



In [115]:
data['domain'].unique()

array(['software-development', nan, 'front-end', 'back-end', 'mobile',
       'analytics', 'full-stack', 'other', 'devops-sre', 'data-science',
       'cyber-security', 'ai-ml', 'qa', 'sales', 'finance', 'marketing',
       'design', 'hr', 'customer-support', 'management'], dtype=object)

In [118]:
for index, row in data[data['domain'].isna()].iterrows():#lisame valdkonna kõikjale, kus domain on NaN
  name=row['job_name']
  if re.search('software engineer|tarkvaraarendaja',name):
    print('replaced ',str(index))
    data.loc[index,'domain']='software-development'
  elif re.search('front-end|frontend|react',name):
    data.loc[index,'domain']='front-end'
  elif re.search('back-end|backend',name):
    data.loc[index,'domain']='back-end'
  elif re.search('android|ios',name):
    data.loc[index,'domain']='mobile'
  elif re.search('devops',name):
    data.loc[index,'domain']='devops-sre'
  elif re.search('data scientist',name):
    data.loc[index,'domain']='data-science'

  

In [119]:
data[data['domain'].isna()]

,company_name,job_name,job_link,date_added,domain
6,nexd,"full-stack developer (php, vue.js)",https://www.nexd.com/career/full-stack-developer/,2024-03-26 00:00:00,NaN
27,nexd,"full-stack developer (php, vue.js)",https://www.nexd.com/career/full-stack-developer/,2024-03-26 00:00:00,NaN
29,novater solutions,full-stack developer,https://novater.com/careers/full-stack-developer/,2024-03-26 00:00:00,NaN
30,seb,full-stack software developer for baltic inter...,https://jobs.eu.lever.co/seb/2b1135be-e2e0-4f7...,2024-03-26 00:00:00,NaN
31,creditstar,full stack developer,https://apply.workable.com/j/9c81ddebd2,2024-03-26 00:00:00,NaN
...,...,...,...,...,...
737,luminor,senior compliance officer (assurance unit),https://luminorbank.teamtailor.com/jobs/354120...,2024-04-02 00:00:00,NaN
738,cybernetica,post-quantum cryptography engineer,https://cyber.teamdash.com/p/job/iqn79wzx/post...,2024-04-02 00:00:00,NaN
739,cybernetica,researcher,https://cyber.teamdash.com/p/job/pagseczx/rese...,2024-04-02 00:00:00,NaN
740,cybernetica,data architect,https://cyber.teamdash.com/p/job/knvredgr/data...,2024-04-02 00:00:00,NaN
